In [1]:
import kineticstoolkit.lab as ktk
import pandas as pd
# Set an interactive backend, not required if already enabled in Spyder
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Relative path to the CSV file
Subject2_SRdata_path = r"../SRrawdata/Subject2/002_f_2024-10-28_14h24.22.400.csv"

# Read the CSV file into a DataFrame
Subject2_SRdata = pd.read_csv(Subject2_SRdata_path)
# Read the C3D fiel into a Dictionary
Subject2_MCdata_path = r"../MCrawdata/Subject2/Ex_V2_S2.c3d"

# Read the C3D fiel into a Dictionary
Subject2_MCdata = ktk.read_c3d(Subject2_MCdata_path)['Points']

UserWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\kineticstoolkit\files.py:519] In the specified file, points are expressed in mm. They were automatically converted to meters by scaling them by 0.001. Please note that if this file also contains calculated values such as angles, powers, etc., they were also (wrongly) scaled by 0.001. Consult https://kineticstoolkit.uqam.ca/doc/api/ktk.read_c3d.html for more information. You can mute this warning by explicitely setting `convert_point_unit` to either True or False.


In [3]:
Subject2_MCdata.data.keys()

dict_keys(['PEN', 'R_H', 'L_H', 'L_S', 'R_S', 'R_L_E', 'R_M_E', 'R_L_W', 'R_M_W', 'R_L_H', 'R_M_H', 'R_F_S', 'R_B_S', 'Back'])

In [4]:
interconnections = dict()

interconnections["Hand"] = {
    "Color": (0, 0.5, 1),  # In (R,G,B) format (here, greenish blue)
    "Links": [  # List of lines that span lists of markers
        ["R_L_H", "R_M_H", "R_M_W", "R_L_W"],
        ['R_L_H','R_L_W']
    ],
}

interconnections["Thorax"] = {
    "Color": (0.5, 1, 0.5),
    "Links": [
        ["R_S", "L_S","Back"],
        ['R_S','Back']
    ],
}

interconnections["Belly"] = {
    "Color": (0.5, 1, 0.5),
    "Links": [
        ["R_H", "L_H","Back"],
        ['R_H','Back']
    ],
}


interconnections["Arm"] = {
    "Color": (1, 0.5, 1),
    "Links": [
        ['R_L_E', "R_L_W","R_M_W",'R_M_E'],
        ['R_L_E',"R_S",'R_M_E'],
        ['R_L_E','R_M_E'],
    ],
}



In [5]:
# p = ktk.Player()
# p.set_contents(Subject2_MCdata)
# p.set_interconnections(interconnections)
# p.up = "z"
# p.anterior = "-y"
# p.set_view("back")
# p.play()

In [6]:
# Assuming SRdata is your DataFrame
# Initialize variables to store trial count and start/end times
trial_count = 0
start_end_times = []

# Iterate through each row in the 'moving_mouse.time' column
for i, row in Subject2_SRdata['moving_mouse.time'].items():
    if pd.notna(row):
        # Increment trial count
        trial_count += 1
        
        # Convert the row to a list and get the first and last element
        time_series = eval(row)  # Using eval to convert string representation of lists to actual lists
        start_time = time_series[0]
        end_time = time_series[-1]
        
        # Store the start and end times
        start_end_times.append((start_time, end_time))

Test the Angle and Angle Velocity we got form ktk

In [7]:
# Read the C3D fiel into a Dictionary
Subject2_R1_path = r"../MCrawdata/Subject2/Ex_V2_R1_S20001.c3d"
Subject2_R2_path = r"../MCrawdata/Subject2/Ex_V2_R2_S20002.c3d"



# Read the C3D fiel into a Dictionary
Subject2_R1 = ktk.read_c3d(Subject2_R1_path)['Points']
Subject2_R2 = ktk.read_c3d(Subject2_R2_path)['Points']

In [8]:
# p = ktk.Player()
# p.set_contents(Subject2_R2 )
# p.set_interconnections(interconnections)
# p.up = "z"
# p.anterior = "-y"
# p.set_view("back")
# p.play()

In [9]:

Subject2_MCdata = Subject2_MCdata.ui_edit_events()

In [10]:
Subject2_MCdata.events

[TimeSeriesEvent(time=np.float64(17.034045698924714), name='Expstart')]

In [11]:
index_expstart = Subject2_MCdata.get_index_at_event('Expstart')

In [12]:
Subject2_MCdata.time[index_expstart]

np.float64(17.033333333333335)

In [13]:
# First start time
first_start_time = start_end_times[0][0]-Subject2_MCdata.time[index_expstart]

# Sync all times by subtracting the first start time
synced_start_end_times = [(start - first_start_time, end - first_start_time) for start, end in start_end_times]

In [52]:
# Set up the plot
plt.figure(figsize=(10, 2))

# Plot each event as a line on the timeline
for (start, end) in synced_start_end_times:
    plt.plot([start, end], [0, 0], marker='|', color='b', linewidth=1)

# Label the axes
plt.xlabel("Time (s)")
plt.yticks([])  # Hide y-axis labels since we don't need them
plt.title("Event Timeline")

# Add grid for better visibility
plt.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

Subject2_MCdata.plot(['PEN'])

# Show the plot
plt.show()

## Create Local corrdinate system

### Arm coordinate system

In [15]:
def get_frames(markers):
    frames = ktk.TimeSeries(time=markers.time)

    # ## Global
    # The global coordinate system was recorded with:
    #            z up, y forward and x to the right.
    # The joint coordinates are in y up, x forward, and z to the right.
    # Let's create a frame that captures this.
    global_transform = ktk.geometry.create_transforms(
        seq="xy", angles=[[-90, -90]], degrees=True
    )

    # ## pelvis
    pelvis_origin = (markers.data["R_H"] + markers.data["L_H"]) / 2
    pelvis_z = markers.data["L_H"] - markers.data["R_H"]
    pelvis_yz = markers.data["Back"] - pelvis_origin

    frames.data["Pelvis"] = ktk.geometry.create_frames(
        origin=pelvis_origin, z=pelvis_z, yz=pelvis_yz
    )

    # ## Thorax
    thorax_origin = (markers.data["L_S"] + markers.data["R_S"]) / 2
    thorax_z = markers.data["L_S"] - markers.data["R_S"]
    thorax_yz = thorax_origin - markers.data["Back"]

    frames.data["Thorax"] = ktk.geometry.create_frames(
        origin=thorax_origin, z=thorax_z, yz=thorax_yz
    )

    # ## Upper arms
    # ### Right upper arm
    r_arm_origin = markers.data["R_S"]
    r_arm_y = markers.data["R_S"] -0.5*(markers.data["R_L_E"] + markers.data["R_M_E"]) 
    r_arm_yz = markers.data["R_L_E"] - markers.data["R_M_E"]

    frames.data["R_Arm"] = ktk.geometry.create_frames(
        origin=r_arm_origin, y=r_arm_y, yz=r_arm_yz
    )


    # ## Forearms
    # ### Right forearm
    r_forearm_origin = markers.data["R_M_W"]
    r_forearm_y = 0.5*(markers.data["R_L_E"] + markers.data["R_M_E"])  - 0.5*(markers.data["R_L_W"]+markers.data["R_M_W"])
    r_forearm_yz = markers.data["R_M_W"] - markers.data["R_L_W"]

    frames.data["R_Forearm"] = ktk.geometry.create_frames(
        origin=r_forearm_origin, y=r_forearm_y, yz=r_forearm_yz
    )

    # ## Hands
    # ### Right hand
    r_hand_origin = markers.data["R_L_W"]
    r_hand_z = markers.data["R_L_W"] - markers.data["R_M_W"]
    r_hand_yz = (markers.data["R_M_H"] + markers.data["R_L_H"]) / 2 - 0.5*(markers.data["R_L_W"]+markers.data["R_M_W"])
    

    frames.data["R_Hand"] = ktk.geometry.create_frames(
        origin=r_hand_origin, z=r_hand_z, yz=r_hand_yz
    )

    return frames, global_transform

In [16]:
framesV2,global_transformV2 = get_frames(Subject2_MCdata)

RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\kineticstoolkit\geometry.py:592] invalid value encountered in divide


In [17]:
def get_homogeneous_angle(frames, global_transform):
    homogeneous = ktk.TimeSeries(time=frames.time)

    if len(frames.time) == 0:
        nan_array = np.full((0, 4, 4), np.nan)
        for key in [
            "pelvis",
            "pelvis_to_thorax",
            "thorax_to_r_arm",
            "R_arm_to_forearm",
            "R_forearm_to_hand",
        ]:
            homogeneous.data[key] = nan_array
        return homogeneous
    try:
        homogeneous.data["pelvis"] = ktk.geometry.get_local_coordinates(
            frames.data["Pelvis"], global_transform
        )
        homogeneous.data["pelvis_to_thorax"] = ktk.geometry.get_local_coordinates(
            frames.data["Thorax"], frames.data["Pelvis"]
        )
        homogeneous.data["thorax_to_r_arm"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Arm"], frames.data["Thorax"]
        )
        homogeneous.data["R_arm_to_forearm"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Forearm"], frames.data["R_Arm"]
        )
        homogeneous.data["R_forearm_to_hand"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Hand"], frames.data["R_Forearm"]
        )

    except Exception as e:
        print(f"An error occurred: {e}")
        
    return homogeneous

In [18]:
get_homogeneous_angle(framesV2,global_transformV2)

TimeSeries with attributes:
         time: <array of shape (74550,)>
         data: <dict with 5 entries>
    time_info: {'Unit': 's'}
    data_info: {}
       events: []

In [19]:

homogeneous = ktk.TimeSeries(time=Subject2_MCdata.time)

global_transform = ktk.geometry.create_transforms(
        seq="xy", angles=[[-90, -90]], degrees=True
    )
homogeneous.data["pelvis"] = ktk.geometry.get_local_coordinates(
    framesV2.data["Pelvis"], global_transform
)
homogeneous.data["pelvis_to_thorax"] = ktk.geometry.get_local_coordinates(
    framesV2.data["Thorax"], framesV2.data["Pelvis"]
        )
homogeneous.data["thorax_to_r_arm"] = ktk.geometry.get_local_coordinates(
    framesV2.data["R_Arm"], framesV2.data["Thorax"]
        )
homogeneous.data["R_arm_to_forearm"] = ktk.geometry.get_local_coordinates(
    framesV2.data["R_Forearm"], framesV2.data["R_Arm"]
        )
homogeneous.data["R_forearm_to_hand"] = ktk.geometry.get_local_coordinates(
            framesV2.data["R_Hand"], framesV2.data["R_Forearm"]
        )

In [53]:
euler = ktk.TimeSeries(time=framesV2.time)
euler.data["Pelvis"] = ktk.geometry.get_angles(
            homogeneous.data["pelvis"], "YZX", degrees=True
        )
euler.data["Thorax"] = ktk.geometry.get_angles(
            homogeneous.data["pelvis_to_thorax"], "YZX", degrees=True
        )
euler.data["R_Arm"] = ktk.geometry.get_angles(
            homogeneous.data["thorax_to_r_arm"], "ZXY", degrees=True
        )
euler.data["R_Forearm"] = ktk.geometry.get_angles(
            homogeneous.data["R_arm_to_forearm"], "ZXY", degrees=True
        )
euler.data["R_Hand"] = ktk.geometry.get_angles(
            homogeneous.data["R_forearm_to_hand"], "ZXY", degrees=True
        )


In [54]:
euler_vels = ktk.filters.deriv(euler)
euler_vels.resample(euler.time, kind="cubic", in_place=True)

TimeSeries with attributes:
         time: <array of shape (74550,)>
         data: <dict with 5 entries>
    time_info: {'Unit': 's'}
    data_info: {}
       events: []

In [79]:
euler_vels.time

array([  0.        ,   0.00666667,   0.01333333, ..., 496.98      ,
       496.98666667, 496.99333333])

In [80]:
len(synced_start_end_times)

85

In [71]:
# Calculate the duration of each trial
durations = [end - start for start, end in synced_start_end_times]

# Calculate the mean and standard deviation of the durations
mean_duration = np.mean(durations)
std_duration = np.std(durations)

min_duration = np.min(durations)
max_duration = np.max(durations)

mean_duration, std_duration, min_duration, max_duration

(np.float64(0.7953104058992775),
 np.float64(0.26746434283432674),
 np.float64(0.5805807998403907),
 np.float64(2.1291561000980437))

In [83]:


# Find closest indices in `euler_vels.time` for each trial's start and end time
trial_indices = [
    (
        np.abs(euler_vels.time - start).argmin(),  # Closest index to start time
        np.abs(euler_vels.time - end).argmin()     # Closest index to end time
    )
    for start, end in synced_start_end_times
]

# Use these indices to slice `euler_vels.time` for each trial
trial_segments = [euler_vels.time[start_idx:end_idx+1] for start_idx, end_idx in trial_indices]

# Check the first few segments as a sample
len(trial_segments)

85

In [90]:
# Set up the plot
plt.figure(figsize=(10, 2))
plt.plot(euler_vels.time, euler_vels.data["R_Forearm"],label ='R_Forearm' )


plt.ylabel('deg/s')

Text(0, 0.5, 'deg/s')

In [95]:
euler_vels.data["R_Forearm"][2555][0]

np.float64(0.956566744458999)

### Forearm [0]

In [117]:
plt.figure(figsize=(12, 6))

for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Get the absolute data segment for the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 0]   # Using abs() on the first angle
    trial_time = euler_vels.time[start_idx:end_idx + 1]

    # Set color based on trial range
    if i < 10:
        color = 'grey'  # Familiarization: trials 1–10
        label = 'Familiarization' if i == 0 else ""
    elif 10 <= i < 40:
        color = 'black'  # Baseline: trials 11–40
        label = 'Baseline' if i == 10 else ""
    else:
        color = 'red'  # Rotation: trials 41–85
        label = 'Rotation' if i == 40 else ""

        # Point 1: 60 indices after start_idx 400ms 0.4s
    point1_idx = start_idx + 60 if start_idx + 30 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 0]

    # Point 2: Maximum value in the trial
    max_idx_in_trial = np.argmax(np.abs(trial_data))  # Index relative to trial_data
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]

        # Plot the trial data segment for context
    plt.plot(trial_time, trial_data,  color=color, label=label, alpha=0.5)

    # Plot Point 1
    plt.plot(point1_time, point1_value, 'o', color='blue', label='Move on set 400ms' if i == 0 else "")

    # Plot Point 2 (maximum value)
    plt.plot(max_time, max_value, 's', color='red', label='Max Value' if i == 0 else "")

# Add labels and legend
plt.ylabel('deg/s (absolute)')
plt.title('Absolute R_Forearm Angle Across Phases')
plt.legend(loc='upper right')
plt.show()

In [118]:
plt.figure(figsize=(12, 6))

for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Get the absolute data segment for the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 0]   # Using abs() on the first angle
    trial_time = euler_vels.time[start_idx:end_idx + 1]

    # Set color based on trial range
    if i < 10:
        color = 'grey'  # Familiarization: trials 1–10
        label = 'Familiarization' if i == 0 else ""
    elif 10 <= i < 40:
        color = 'black'  # Baseline: trials 11–40
        label = 'Baseline' if i == 10 else ""
    else:
        color = 'red'  # Rotation: trials 41–85
        label = 'Rotation' if i == 40 else ""

        # Point 1: 60 indices after start_idx 400ms 0.4s
    point1_idx = start_idx + 30 if start_idx + 30 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 0]

    # Point 2: Maximum value in the trial
    max_idx_in_trial = np.argmax(np.abs(trial_data))  # Index relative to trial_data
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]

        # Plot the trial data segment for context
    plt.plot(trial_time, trial_data,  color=color, label=label, alpha=0.5)

    # Plot Point 1
    plt.plot(point1_time, point1_value, 'o', color='blue', label='Move on set 200ms' if i == 0 else "")

    # Plot Point 2 (maximum value)
    plt.plot(max_time, max_value, 's', color='red', label='Max Value' if i == 0 else "")

# Add labels and legend
plt.ylabel('deg/s (absolute)')
plt.title('Absolute R_Forearm Angle Across Phases')
plt.legend(loc='upper right')
plt.show()

In [230]:
# Initialize an empty list to hold the results
Angvel_R_Forearm0 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["R_Forearm"][point_indices[j], 0]
        point_euler = euler.data["R_Forearm"][point_indices[j], 0]

        # Append the results for each point
        Angvel_R_Forearm0.append({
            "Angle Name": f"R_Forearm[0] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_R_Forearm0 = pd.DataFrame(Angvel_R_Forearm0)


In [231]:
Angvel_R_Forearm0

,Angle Name,Trial ID,Time,Value,Euler
0,R_Forearm[0] point 1,1,17.186667,1.561345,84.356123
1,R_Forearm[0] point 2,1,17.346667,0.572812,84.432207
2,R_Forearm[0] point 3,1,17.506667,1.866088,84.547105
3,R_Forearm[0] point 4,1,17.660000,-0.777655,84.753455
4,R_Forearm[0] point 5,1,17.820000,-20.344830,83.689561
...,...,...,...,...,...
845,R_Forearm[0] point 6,85,478.220000,-1.344128,86.906309
846,R_Forearm[0] point 7,85,478.280000,-1.322552,86.870474
847,R_Forearm[0] point 8,85,478.346667,-2.165617,86.762579
848,R_Forearm[0] point 9,85,478.406667,-8.933067,86.496018


### Forearm [2]

In [232]:
# Initialize an empty list to hold the results
Angvel_R_Forearm2 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["R_Forearm"][point_indices[j], 2]
        point_euler = euler.data["R_Forearm"][point_indices[j], 2]

        # Append the results for each point
        Angvel_R_Forearm2.append({
            "Angle Name": f"R_Forearm[2] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_R_Forearm2 = pd.DataFrame(Angvel_R_Forearm2)


In [124]:
plt.figure(figsize=(12, 6))

for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Get the absolute data segment for the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 2]   # Using abs() on the third angle
    trial_time = euler_vels.time[start_idx:end_idx + 1]

    # Set color based on trial range
    if i < 10:
        color = 'grey'  # Familiarization: trials 1–10
        label = 'Familiarization' if i == 0 else ""
    elif 10 <= i < 40:
        color = 'black'  # Baseline: trials 11–40
        label = 'Baseline' if i == 10 else ""
    else:
        color = 'red'  # Rotation: trials 41–85
        label = 'Rotation' if i == 40 else ""

        # Point 1: 60 indices after start_idx 400ms 0.4s
    point1_idx = start_idx + 30 if start_idx + 30 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 2]

    # Point 2: Maximum value in the trial
    max_idx_in_trial = np.argmax(np.abs(trial_data))  # Index relative to trial_data
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]

        # Plot the trial data segment for context
    plt.plot(trial_time, trial_data,  color=color, label=label, alpha=0.5)

    # Plot Point 1
    plt.plot(point1_time, point1_value, 'o', color='blue', label='Move on set 200ms' if i == 0 else "")

    # Plot Point 2 (maximum value)
    plt.plot(max_time, max_value, 's', color='red', label='Max Value' if i == 0 else "")

# Add labels and legend
plt.ylabel('deg/s (absolute)')
plt.title('Absolute R_Forearm Angle[2] Across Phases')
plt.legend(loc='upper right')
plt.show()

In [125]:
plt.figure(figsize=(12, 6))

for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Get the absolute data segment for the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 2]   # Using abs() on the third angle
    trial_time = euler_vels.time[start_idx:end_idx + 1]

    # Set color based on trial range
    if i < 10:
        color = 'grey'  # Familiarization: trials 1–10
        label = 'Familiarization' if i == 0 else ""
    elif 10 <= i < 40:
        color = 'black'  # Baseline: trials 11–40
        label = 'Baseline' if i == 10 else ""
    else:
        color = 'red'  # Rotation: trials 41–85
        label = 'Rotation' if i == 40 else ""

        # Point 1: 60 indices after start_idx 400ms 0.4s
    point1_idx = start_idx + 60 if start_idx + 60 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 2]

    # Point 2: Maximum value in the trial
    max_idx_in_trial = np.argmax(np.abs(trial_data))  # Index relative to trial_data
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]

        # Plot the trial data segment for context
    plt.plot(trial_time, trial_data,  color=color, label=label, alpha=0.5)

    # Plot Point 1
    plt.plot(point1_time, point1_value, 'o', color='blue', label='Move on set 400ms' if i == 0 else "")

    # Plot Point 2 (maximum value)
    plt.plot(max_time, max_value, 's', color='red', label='Max Value' if i == 0 else "")

# Add labels and legend
plt.ylabel('deg/s (absolute)')
plt.title('Absolute R_Forearm Angle[2] Across Phases')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# # Initialize list to store data for each trial's points
# Angvel_R_Forearm2 = []

# # Loop through each trial and extract point 1 and point 2 values
# for i, (start_idx, end_idx) in enumerate(trial_indices):
#     # Point 1: 21 indices after start_idx
#     point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
#     point1_time = euler_vels.time[point1_idx]
#     point1_value = euler_vels.data["R_Forearm"][point1_idx, 2]
#     point1_euler = euler.data["R_Forearm"][point1_idx, 2]
    
#     # Point 2: Maximum value in the trial
#     trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 2]
#     trial_time = euler_vels.time[start_idx:end_idx + 1]
#     max_idx_in_trial = np.argmax(np.abs(trial_data))
#     max_time = trial_time[max_idx_in_trial]
#     max_value = trial_data[max_idx_in_trial]
#     max_value_euler = euler.data["R_Forearm"][max_idx_in_trial , 2]
    
#     # Add data for point 1
#     Angvel_R_Forearm2.append({
#         "Angle Name": "R_Forearm[2] move onset",
#         "Trial ID": i + 1,
#         "Time": point1_time,
#         "Value": point1_value,
#         "Euler": point1_euler
#     })
    
#     # Add data for point 2
#     Angvel_R_Forearm2.append({
#         "Angle Name": "R_Forearm[2] max vel",
#         "Trial ID": i + 1,
#         "Time": max_time,
#         "Value": max_value,
#         "Euler": max_value_euler
#     })

# # Convert the list to a DataFrame
# Angvel_R_Forearm2 = pd.DataFrame(Angvel_R_Forearm2)


### Arm[0]

In [238]:
# Initialize list to store data for each trial's points
Angvel_R_arm0 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["R_Arm"][point_indices[j], 0]
        point_euler = euler.data["R_Arm"][point_indices[j], 0]

        # Append the results for each point
        Angvel_R_arm0.append({
            "Angle Name": f"R_arm[0] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_R_arm0 = pd.DataFrame(Angvel_R_arm0)


### Arm[2]

In [237]:
# Initialize list to store data for each trial's points
Angvel_R_arm2 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["R_Arm"][point_indices[j], 2]
        point_euler = euler.data["R_Arm"][point_indices[j], 2]

        # Append the results for each point
        Angvel_R_arm2.append({
            "Angle Name": f"R_arm[2] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })


# Convert the list to a DataFrame
Angvel_R_arm2 = pd.DataFrame(Angvel_R_arm2)


### Hand[0]

In [239]:
# Initialize list to store data for each trial's points
Angvel_R_hand0 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["R_Hand"][point_indices[j], 0]
        point_euler = euler.data["R_Hand"][point_indices[j], 0]

        # Append the results for each point
        Angvel_R_hand0.append({
            "Angle Name": f"R_hand[0] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })


# Convert the list to a DataFrame
Angvel_R_hand0 = pd.DataFrame(Angvel_R_hand0)


### Throax

In [248]:
# Initialize list to store data for each trial's points
Angvel_throax0 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Thorax"][point_indices[j], 0]
        point_euler = euler.data["Thorax"][point_indices[j], 0]

        # Append the results for each point
        Angvel_throax0.append({
            "Angle Name": f"Throax[0] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_throax0 = pd.DataFrame(Angvel_throax0)


In [249]:
# Initialize list to store data for each trial's points
Angvel_throax1 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Thorax"][point_indices[j], 1]
        point_euler = euler.data["Thorax"][point_indices[j], 1]

        # Append the results for each point
        Angvel_throax1.append({
            "Angle Name": f"Throax[1] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_throax1 = pd.DataFrame(Angvel_throax1)


In [250]:
# Initialize list to store data for each trial's points
Angvel_throax2 = []

# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Thorax"][point_indices[j], 2]
        point_euler = euler.data["Thorax"][point_indices[j], 2]

        # Append the results for each point
        Angvel_throax2.append({
            "Angle Name": f"Throax[2] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_throax2 = pd.DataFrame(Angvel_throax2)

### Pelvis

In [252]:
# Initialize list to store data for each trial's points
Angvel_pelvis0 = []


# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Pelvis"][point_indices[j], 0]
        point_euler = euler.data["Pelvis"][point_indices[j], 0]

        # Append the results for each point
        Angvel_pelvis0.append({
            "Angle Name": f"Pelvis[0] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_pelvis0 = pd.DataFrame(Angvel_pelvis0)


In [253]:
# Initialize list to store data for each trial's points
Angvel_pelvis1 = []


# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Pelvis"][point_indices[j], 1]
        point_euler = euler.data["Pelvis"][point_indices[j], 1]

        # Append the results for each point
        Angvel_pelvis1.append({
            "Angle Name": f"Pelvis[1] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_pelvis1= pd.DataFrame(Angvel_pelvis1)


In [254]:
# Initialize list to store data for each trial's points
Angvel_pelvis2 = []


# Loop through each trial and extract 10 point values equally from start and end_idx 
for i, (start_idx, end_idx) in enumerate(trial_indices):
    trial_length = end_idx - start_idx

    # Calculate the indices for the 10 points in a vectorized way
    point_indices = np.clip(np.linspace(start_idx, end_idx, num=11, dtype=int), start_idx, end_idx)

    # Extract the corresponding time, value, and euler for each point
    for j in range(1, 11):  # points 1 to 10
        point_time = euler_vels.time[point_indices[j]]
        point_value = euler_vels.data["Pelvis"][point_indices[j], 2]
        point_euler = euler.data["Pelvis"][point_indices[j], 2]

        # Append the results for each point
        Angvel_pelvis2.append({
            "Angle Name": f"Pelvis[2] point {j}",
            "Trial ID": i + 1,
            "Time": point_time,
            "Value": point_value,
            "Euler": point_euler
        })

# Convert the list to a DataFrame
Angvel_pelvis2= pd.DataFrame(Angvel_pelvis2)


In [ ]:
Angvel_R_Forearm0
Angvel_R_Forearm2
Angvel_R_arm0
Angvel_R_arm2
Angvel_R_hand0
Angvel_throax0 
Angvel_throax1 
Angvel_throax2
Angvel_pelvis0
Angvel_pelvis1
Angvel_pelvis2

In [255]:
# List of DataFrames
all_joints_list = [
    Angvel_R_Forearm0, Angvel_R_Forearm2, Angvel_R_arm0, Angvel_R_arm2,
    Angvel_R_hand0, Angvel_throax0, Angvel_throax1, Angvel_throax2,
    Angvel_pelvis0, Angvel_pelvis1, Angvel_pelvis2
]

# Concatenate all data frames
all_joints_df = pd.concat(all_joints_list, ignore_index=True)

# Ensure the data is sorted by 'Trial ID'
all_joints_df = all_joints_df.sort_values(by=["Trial ID", "Angle Name"]).reset_index(drop=True)



In [303]:
all_joints_df

,Angle Name,Trial ID,Time,Value,Euler
0,Pelvis[0] point 1,1,17.186667,-0.266223,129.889728
1,Pelvis[0] point 10,1,18.613333,0.847657,129.201517
2,Pelvis[0] point 2,1,17.346667,0.504755,129.845117
3,Pelvis[0] point 3,1,17.506667,0.066762,129.869431
4,Pelvis[0] point 4,1,17.660000,-0.546493,129.859971
...,...,...,...,...,...
9345,Throax[2] point 5,85,478.153333,1.864997,-4.542233
9346,Throax[2] point 6,85,478.220000,3.423972,-4.372681
9347,Throax[2] point 7,85,478.280000,0.743898,-4.366411
9348,Throax[2] point 8,85,478.346667,-2.835748,-4.454536


In [306]:
# Step 1: Filter trials 30 to 40
Rotation_data = all_joints_df[(all_joints_df['Trial ID'] >= 41) & (all_joints_df['Trial ID'] <= 85)]

# # Step 2: Group by 'Angle Name' and calculate the mean value for each angle
# base_vel= Rotation_data.groupby('Angle Name')['Value'].mean().reset_index()
# base_euler = Rotation_data.groupby('Angle Name')['Euler'].mean().reset_index()

# # Step 3: Rename the column for clarity if needed
# base_vel.rename(columns={'Value': 'Base Value'}, inplace=True)
# base_euler.rename(columns={'Euler': 'Base Euler'}, inplace=True)


In [316]:
Rotation_data.sort_values(by=["Trial ID","Time", "Angle Name"]).reset_index(drop=True)

,Angle Name,Trial ID,Time,Value,Euler
0,Pelvis[0] point 1,41,247.646667,-0.226769,131.050504
1,Pelvis[1] point 1,41,247.646667,0.103009,-3.759477
2,Pelvis[2] point 1,41,247.646667,0.031035,87.178806
3,R_Forearm[0] point 1,41,247.646667,-0.807418,91.194983
4,R_Forearm[2] point 1,41,247.646667,5.315962,-52.769433
...,...,...,...,...,...
4945,R_arm[2] point 10,85,478.473333,-36.361582,-137.504083
4946,R_hand[0] point 10,85,478.473333,42.672508,176.037068
4947,Throax[0] point 10,85,478.473333,9.304835,13.336283
4948,Throax[1] point 10,85,478.473333,1.859987,53.980471


In [321]:
Angvel_R_Forearm0

,Angle Name,Trial ID,Time,Value,Euler
0,R_Forearm[0] point 1,1,17.186667,1.561345,84.356123
1,R_Forearm[0] point 2,1,17.346667,0.572812,84.432207
2,R_Forearm[0] point 3,1,17.506667,1.866088,84.547105
3,R_Forearm[0] point 4,1,17.660000,-0.777655,84.753455
4,R_Forearm[0] point 5,1,17.820000,-20.344830,83.689561
...,...,...,...,...,...
845,R_Forearm[0] point 6,85,478.220000,-1.344128,86.906309
846,R_Forearm[0] point 7,85,478.280000,-1.322552,86.870474
847,R_Forearm[0] point 8,85,478.346667,-2.165617,86.762579
848,R_Forearm[0] point 9,85,478.406667,-8.933067,86.496018


In [ ]:
# Trial1_Value = {
#     "R_Forearm[0]": Angvel_R_Forearm0.loc[Angvel_R_Forearm0['Trial ID'] == 1, 'Value'].values,
#     "R_Forearm[2]": Angvel_R_Forearm2.loc[Angvel_R_Forearm2['Trial ID'] == 1, 'Value'].values,
#     "R_arm[0]": Angvel_R_arm0.loc[Angvel_R_arm0['Trial ID'] == 1, 'Value'].values,
#     "R_arm[2]": Angvel_R_arm2.loc[Angvel_R_arm2['Trial ID'] == 1, 'Value'].values,
#     "R_hand[0]": Angvel_R_hand0.loc[Angvel_R_hand0['Trial ID'] == 1, 'Value'].values,
#     "Throax[0]": Angvel_throax0.loc[Angvel_throax0['Trial ID'] == 1, 'Value'].values,
#     "Throax[1]": Angvel_throax1.loc[Angvel_throax1['Trial ID'] == 1, 'Value'].values,
#     "Throax[2]": Angvel_throax2.loc[Angvel_throax2['Trial ID'] == 1, 'Value'].values,
#     "Pelvis[0]": Angvel_pelvis0.loc[Angvel_pelvis0['Trial ID'] == 1, 'Value'].values,
#     "Pelvis[1]": Angvel_pelvis1.loc[Angvel_pelvis1['Trial ID'] == 1, 'Value'].values,
#     "Pelvis[2]": Angvel_pelvis2.loc[Angvel_pelvis2['Trial ID'] == 1, 'Value'].values
# }

# Trial1_Eluer ={
#     "R_Forearm[0]": Angvel_R_Forearm0.loc[Angvel_R_Forearm0['Trial ID'] == 1, 'Euler'].values,
#     "R_Forearm[2]": Angvel_R_Forearm2.loc[Angvel_R_Forearm2['Trial ID'] == 1, 'Euler'].values,
#     "R_arm[0]": Angvel_R_arm0.loc[Angvel_R_arm0['Trial ID'] == 1, 'Euler'].values,
#     "R_arm[2]": Angvel_R_arm2.loc[Angvel_R_arm2['Trial ID'] == 1, 'Euler'].values,
#     "R_hand[0]": Angvel_R_hand0.loc[Angvel_R_hand0['Trial ID'] == 1, 'Euler'].values,
#     "Throax[0]": Angvel_throax0.loc[Angvel_throax0['Trial ID'] == 1, 'Euler'].values,
#     "Throax[1]": Angvel_throax1.loc[Angvel_throax1['Trial ID'] == 1, 'Euler'].values,
#     "Throax[2]": Angvel_throax2.loc[Angvel_throax2['Trial ID'] == 1, 'Euler'].values,
#     "Pelvis[0]": Angvel_pelvis0.loc[Angvel_pelvis0['Trial ID'] == 1, 'Euler'].values,
#     "Pelvis[1]": Angvel_pelvis1.loc[Angvel_pelvis1['Trial ID'] == 1, 'Euler'].values,
#     "Pelvis[2]": Angvel_pelvis2.loc[Angvel_pelvis2['Trial ID'] == 1, 'Euler'].values
# }

# Trial1_Value_df = pd.DataFrame(Trial1_Value)
# Trial1_Eluer_df = pd.DataFrame(Trial1_Eluer)
# Trial1_combined_df = pd.concat([Trial1_Value_df, Trial1_Eluer_df], ignore_index=True)
# cov_matrix =Trial1_combined_df.cov()
# det_value = np.linalg.det(cov_matrix)

In [344]:
# Initialize an empty list to store the determinant values for each trial
det_values = []

# Loop through Trial IDs from 1 to 85
for trial_id in range(1, 85):
    Trial_Value = {
        "R_Forearm[0]": Angvel_R_Forearm0.loc[Angvel_R_Forearm0['Trial ID'] == trial_id, 'Value'].values,
        "R_Forearm[2]": Angvel_R_Forearm2.loc[Angvel_R_Forearm2['Trial ID'] == trial_id, 'Value'].values,
        "R_arm[0]": Angvel_R_arm0.loc[Angvel_R_arm0['Trial ID'] == trial_id, 'Value'].values,
        "R_arm[2]": Angvel_R_arm2.loc[Angvel_R_arm2['Trial ID'] == trial_id, 'Value'].values,
        "R_hand[0]": Angvel_R_hand0.loc[Angvel_R_hand0['Trial ID'] == trial_id, 'Value'].values,
        "Throax[0]": Angvel_throax0.loc[Angvel_throax0['Trial ID'] == trial_id, 'Value'].values,
        "Throax[1]": Angvel_throax1.loc[Angvel_throax1['Trial ID'] == trial_id, 'Value'].values,
        "Throax[2]": Angvel_throax2.loc[Angvel_throax2['Trial ID'] == trial_id, 'Value'].values,
        "Pelvis[0]": Angvel_pelvis0.loc[Angvel_pelvis0['Trial ID'] == trial_id, 'Value'].values,
        "Pelvis[1]": Angvel_pelvis1.loc[Angvel_pelvis1['Trial ID'] == trial_id, 'Value'].values,
        "Pelvis[2]": Angvel_pelvis2.loc[Angvel_pelvis2['Trial ID'] == trial_id, 'Value'].values
    }

    Trial_Euler = {
        "R_Forearm[0]": Angvel_R_Forearm0.loc[Angvel_R_Forearm0['Trial ID'] == trial_id, 'Euler'].values,
        "R_Forearm[2]": Angvel_R_Forearm2.loc[Angvel_R_Forearm2['Trial ID'] == trial_id, 'Euler'].values,
        "R_arm[0]": Angvel_R_arm0.loc[Angvel_R_arm0['Trial ID'] == trial_id, 'Euler'].values,
        "R_arm[2]": Angvel_R_arm2.loc[Angvel_R_arm2['Trial ID'] == trial_id, 'Euler'].values,
        "R_hand[0]": Angvel_R_hand0.loc[Angvel_R_hand0['Trial ID'] == trial_id, 'Euler'].values,
        "Throax[0]": Angvel_throax0.loc[Angvel_throax0['Trial ID'] == trial_id, 'Euler'].values,
        "Throax[1]": Angvel_throax1.loc[Angvel_throax1['Trial ID'] == trial_id, 'Euler'].values,
        "Throax[2]": Angvel_throax2.loc[Angvel_throax2['Trial ID'] == trial_id, 'Euler'].values,
        "Pelvis[0]": Angvel_pelvis0.loc[Angvel_pelvis0['Trial ID'] == trial_id, 'Euler'].values,
        "Pelvis[1]": Angvel_pelvis1.loc[Angvel_pelvis1['Trial ID'] == trial_id, 'Euler'].values,
        "Pelvis[2]": Angvel_pelvis2.loc[Angvel_pelvis2['Trial ID'] == trial_id, 'Euler'].values
    }

    Trial_Value_df = pd.DataFrame(Trial_Value)
    Trial_Euler_df = pd.DataFrame(Trial_Euler)
    
    # Combine the DataFrames
    Trial_combined_df = pd.concat([Trial_Value_df, Trial_Euler_df], ignore_index=True)
    
    # Calculate the covariance matrix
    cov_matrix = Trial_combined_df.cov()
    
    # Calculate the determinant of the covariance matrix
    det_value = np.linalg.det(cov_matrix)
    
    # Append the determinant value to the list
    det_values.append(det_value)

# Create a DataFrame for plotting
det_values_df = pd.DataFrame({
    'Trial ID': range(1, 85),
    'Determinant Value': det_values
})

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(det_values_df['Trial ID'], det_values_df['Determinant Value'], marker='o')
plt.title('Determinant of Covariance Matrix by Trial ID')
plt.xlabel('Trial ID')
plt.ylabel('Determinant Value')
plt.grid()
plt.show()

In [356]:
filtered_det_values_df = det_values_df[det_values_df['Trial ID'].between(75, 85)]

# Plotting the filtered data
plt.figure(figsize=(10, 6))
plt.plot(filtered_det_values_df['Trial ID'], filtered_det_values_df['Determinant Value'], marker='o')
plt.title('Determinant of Covariance Matrix for Trial IDs 45 to 80')
plt.xlabel('Trial ID')
plt.ylabel('Determinant Value')
plt.grid()

plt.show()

In [178]:
# Initialize list to store determinants
determinants = []

# Loop through each trial
for trial_id, trial_data in all_joints_df.groupby("Trial ID"):
    # Pivot the data for each trial to get one column per "Angle Name"
    trial_pivot = trial_data.pivot(index="Trial ID", columns="Angle Name", values="Value")
    
    # Drop rows/columns with NaNs if they exist to avoid calculation errors
    trial_pivot = trial_pivot.dropna(axis=1, how='any')
    
    # Ensure there are enough data points to calculate covariance
    if trial_pivot.shape[1] >= 2:  # Check if there are at least two variables after dropping NaNs
        cov_matrix = trial_pivot.cov()
        
        # Calculate the determinant if cov_matrix is valid
        det_value = np.linalg.det(cov_matrix) if np.all(np.isfinite(cov_matrix)) else np.nan
        determinants.append({"Trial ID": trial_id, "Determinant": det_value})

# Convert determinants list to a DataFrame for plotting
det_df = pd.DataFrame(determinants).dropna()  # Drop any NaNs in the determinant column

# Plot the determinants
plt.figure(figsize=(12, 6))
plt.plot(det_df["Trial ID"], det_df["Determinant"], marker='o', linestyle='-')
plt.xlabel("Trial ID")
plt.ylabel("Determinant of Covariance Matrix")
plt.title("Determinant of Covariance Matrix Across Trials")
plt.grid(True)
plt.show()

RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\pandas\core\frame.py:11211] Degrees of freedom <= 0 for slice
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\numpy\lib\_function_base_impl.py:2848] divide by zero encountered in divide
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\numpy\lib\_function_base_impl.py:2848] invalid value encountered in multiply
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\pandas\core\frame.py:11211] Degrees of freedom <= 0 for slice
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\numpy\lib\_function_base_impl.py:2848] divide by zero encountered in divide
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\numpy\lib\_function_base_impl.py:2848] invalid value encountered in multiply
RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\pandas\core\frame.py:11211] Degrees of freedom <= 0 for slice
RuntimeWarning [c:\Users\86153\a

In [ ]:
plt.figure(figsize=(10, 2))
plt.plot(euler_vels.time, euler_vels.data["R_Forearm"],label ='R_Forearm' )


plt.ylabel('deg/s')

In [88]:
plt.figure(figsize=(10, 2))
plt.plot(euler.time, euler.data["R_Hand"],label = "R_Hand" )

plt.title('Example Angle')

plt.ylabel('deg')
plt.legend()
# plt.ylim(-100,100)
plt.show()

In [147]:
plt.figure(figsize=(10, 2))
#plt.plot(euler.time, euler.data["R_Forearm"][:, 2],label ='R_Forearm' )

# plt.plot(euler.time, euler.data["R_Arm"],label = 'R_Arm' )

# plt.plot(euler.time, euler.data["R_Hand"],label = "R_Hand" )

plt.plot(euler.time, euler.data["Thorax"],label = "Thorax" )

# plt.plot(euler.time, euler.data["Pelvis"],label = "Pelvis" )

plt.title('Example Angle')

plt.ylabel('deg')
plt.legend()
# plt.ylim(-100,100)
plt.show()

In [144]:
euler_vels.data

{
       'Pelvis': <array of shape (74550, 3)>
       'Thorax': <array of shape (74550, 3)>
        'R_Arm': <array of shape (74550, 3)>
    'R_Forearm': <array of shape (74550, 3)>
       'R_Hand': <array of shape (74550, 3)>
}